### Import Libraries

In [ ]:
""" Uncomment and run this cell if you donot have these libraries """

# !pip install cv2
# !pip install pandas
# !pip install numpy
# !pip install img2pdf

     |████████████████████████████████| 92kB 4.6MB/s 
  Created wheel for img2pdf: filename=img2pdf-0.3.6-cp36-none-any.whl size=39048 sha256=c51d5595d67500c63931e5b13cb3689fc08b42576c64df2e73df58e91af2618d
  Stored in directory: /root/.cache/pip/wheels/8c/1c/68/d8764ed7988b469446ed181a656166170ec45b6b1f974f0236
Successfully built img2pdf


In [ ]:
import cv2
import pandas as pd
import numpy as np
from PIL import Image
import glob, os
import img2pdf

from google.colab import drive
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

In [ ]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
path = 'gdrive/My Drive/Colab Notebooks'
certi_data_path = 'gdrive/My Drive/Colab Notebooks/final_list_left/'
certi_temp_path = 'gdrive/My Drive/Colab Notebooks/Certificates - NanoDev2020/'

In [ ]:
!ls 'gdrive/My Drive/Colab Notebooks/final_list_left'

'organizers (1).xlsx'
 organziers
 participant_left_pics
'speakers (1).xlsx'
'to be send_certificate num_NanoDev2020-20200705 (6).xlsx'


### Read the Names/Content to be written on the certificate
* The names should be in an excel or csv file for this code
* If you have the data in any other file format, please use the required file reading syntax

In [ ]:
""" Fill in the file path where you have the text to be written on the certificate """
text_file_path = certi_data_path + 'speakers (1).xlsx'
certificate_text = pd.read_excel(text_file_path)

""" Use this if you have csv file """
# certificate_text = pd.read_csv(text_file_path)
certificate_text.head()

,Certificate no.,Full Name,Title
0,20313,H. S. Jatana,Challenges in Development of Photodetectors
1,20314,Yogesh Chauhan,Physics and Modeling of Nano-Transistors
2,20315,D. S. Rawal,GaN HEMT Device Fabrication Technology
3,20316,Amit Singh,Introduction to Silicon High Voltage technology
4,20317,Santanu Mahaptra,Atom-to-Circuit Modeling Technique for 2D MOSFET


In [ ]:
certificate_text.shape

(16, 3)

* Find the appropriate column in the certificate_text dataframe and read the data out of it

Here I present the examples of how to do that.

1. If you have names in a single columns say Participants, then, <br>
    names = np.array(certificate_text['Participants'].dropna()) <br>
    
    
2. If you have names that are spread over different columns, then, <br>
    names_1 = np.array(certificate_text['Name'].dropna()).tolist() <br>
    names_2 = np.array(certificate_text['Name1'].dropna()).tolist() <br>
    names_3 = np.array(certificate_text['Name2'].dropna()).tolist() <br>
    names = names_1 + names_2 + names_3 <br>
    
    
* Do this same for every text column you need on the certificate

In [ ]:
""" Change this according to the data you have """
names = np.array(certificate_text['Full Name'].dropna())
certi_number = np.array(certificate_text['Certificate no.'].dropna())
organization = np.array(certificate_text['Title'].dropna())
len(names), len(certi_number), len(organization)

(16, 16, 16)

In [ ]:
""" Convert all names to Uppercase """
names = [i.upper() for i in names]
organization = [i.upper() for i in organization]
certi_number = [int(i) for i in certi_number]

In [ ]:
""" certi_template -> the png or jpg certificate template """
certi_template = certi_temp_path + 'coa_nanodev20.jpg'

""" certi_output_dir -> the directory where you want to have your certificates generated """
dirName = 'NanoDev Speakers'
if not os.path.exists(path + '/' + dirName):
        os.mkdir(path + '/' + dirName)
        print('directory made successfully')
else:
    print('directory already exists')

certi_output_dir = path + '/' + dirName + '/'
certi_output_dir

directory already exists


'gdrive/My Drive/Colab Notebooks/NanoDev Speakers/'

### Main function to write text on the certificate

In [ ]:
""" Set these parameters for your certificate template 
    Generate single certificate and try out these parameters accordingly
"""
font_size = 2
font_color = (0,0,0)  # RGB channel -> currently set to black
x_coordinate = 0
y_coordinate = 75

font_size_org = 2
x_coordinate_org = 5
y_coordinate_org = -20

font_size_certi = 1.5
x_coordinate_certi = -325
y_coordinate_certi = 405

In [ ]:
def make_certificate(names, certi_template, certi_output_dir):
    count = 0
    for i in range(len(names)):
        name = names[i]
        org = organization[i]
        certi_no = str(certi_number[i])

        if (len(org)) > 32:
            font_size_org = 1.25
        else:
            font_size_org = 2

        if len(org) > 70:
            print(name, org)
            org = ''    

        count += 1
        certi = cv2.imread(certi_template)
        font = cv2.FONT_HERSHEY_PLAIN

        # text 1
        textsize = cv2.getTextSize(name, font, font_size, 2)[0]
        text_x = (certi.shape[1] - textsize[0]) / 2 + x_coordinate
        text_y = (certi.shape[0] + textsize[1]) / 2 - y_coordinate
        text_x = int(text_x)
        text_y = int(text_y)


        # text 2
        textsize_org = cv2.getTextSize(org, font, font_size_org, 2)[0]
        text_x_org = (certi.shape[1] - textsize_org[0]) / 2 + x_coordinate_org
        text_y_org = (certi.shape[0] + textsize_org[1]) / 2 - y_coordinate_org
        text_x_org = int(text_x_org)
        text_y_org = int(text_y_org)

        # text 3
        textsize_certi = cv2.getTextSize(certi_no, font, font_size_certi, 2)[0]
        text_x_certi = (certi.shape[1] - textsize_certi[0]) / 2 + x_coordinate_certi
        text_y_certi = (certi.shape[0] + textsize_certi[1]) / 2 - y_coordinate_certi
        text_x_certi = int(text_x_certi)
        text_y_certi = int(text_y_certi)

        cv2.putText(certi, name, (text_x, text_y ), font, font_size, font_color, 2) # last for thickness
        cv2.putText(certi, org, (text_x_org, text_y_org ), font, font_size_org, font_color, 2) # last for thickness
        cv2.putText(certi, certi_no, (text_x_certi, text_y_certi ), font, font_size_certi, font_color, 2) # last for thickness

        certi_path = certi_output_dir + name +  '_' + certi_no + '.png'
        cv2.imwrite(certi_path,certi)
        print(count, end = ' ')
        print(' Writing certi for ' + name +  '_' + certi_no + ' to ' + certi_path)

In [ ]:
""" Make the funtion call """
make_certificate(names, certi_template, certi_output_dir)

1  Writing certi for H. S. JATANA_20313 to gdrive/My Drive/Colab Notebooks/NanoDev Speakers pdf Certificates/H. S. JATANA_20313.png
2  Writing certi for YOGESH CHAUHAN_20314 to gdrive/My Drive/Colab Notebooks/NanoDev Speakers pdf Certificates/YOGESH CHAUHAN_20314.png
3  Writing certi for D. S. RAWAL_20315 to gdrive/My Drive/Colab Notebooks/NanoDev Speakers pdf Certificates/D. S. RAWAL_20315.png
4  Writing certi for AMIT SINGH_20316 to gdrive/My Drive/Colab Notebooks/NanoDev Speakers pdf Certificates/AMIT SINGH_20316.png
5  Writing certi for SANTANU MAHAPTRA_20317 to gdrive/My Drive/Colab Notebooks/NanoDev Speakers pdf Certificates/SANTANU MAHAPTRA_20317.png
6  Writing certi for JAWAR SINGH_20318 to gdrive/My Drive/Colab Notebooks/NanoDev Speakers pdf Certificates/JAWAR SINGH_20318.png
7  Writing certi for AYAN KARMAKAR_20319 to gdrive/My Drive/Colab Notebooks/NanoDev Speakers pdf Certificates/AYAN KARMAKAR_20319.png
8  Writing certi for ARUN ASHOK_20320 to gdrive/My Drive/Colab Noteboo

In [ ]:
# path2 = 'gdrive/My Drive/Colab Notebooks/NanoDev Speakers/'
# count = 0
# for infile in glob.glob(os.path.join(certi_data_path + 'participant_left_pics/' + '*.*')):
#     count += 1
#     certi_no = infile.split('/')[-1].split('.')[0]
#     name = certificate_text[certificate_text['Certificate no.'] == int(certi_no)]['Full Name'].values[0]
#     name = name.upper()
#     img = Image.open(infile)
#     img = img.resize((170, 170))
#     certi_path = path2 + name + '_' + certi_no + '.png'
#     certi = Image.open(certi_path)
#     certi.paste(img, (1025,210))
#     certi.save(certi_path)
#     print(count, end = ' ')
#     print(' Writing certi with image for ' + name +  '_' + certi_no + ' to ' + certi_path)

In [ ]:
""" certi_output_dir -> the directory where you want to have your certificates generated """
dirName = 'NanoDev Speakers pdf Certificates'
if not os.path.exists(path + '/' + dirName):
        os.mkdir(path + '/' + dirName)
        print('directory made successfully')
else:
    print('directory already exists')

certi_output_dir = path + '/' + dirName + '/'
certi_output_dir

directory already exists


'gdrive/My Drive/Colab Notebooks/NanoDev Speakers pdf Certificates/'

In [ ]:
path2 = 'gdrive/My Drive/Colab Notebooks/NanoDev Speakers/'
pdf_dir = certi_output_dir
count = 0
for infile in glob.glob(os.path.join(path2 + '*.png')):
    count += 1
    name = infile.split('/')[-1][:-4]
    img = Image.open(infile)
    pdf_path = pdf_dir + name + '.pdf'

    pdf_bytes = img2pdf.convert(img.filename)
    file = open(pdf_path, "wb") 
    file.write(pdf_bytes) 
  
    img.close() 
    file.close() 
    print(count, end = ' ')
    print('PDF written for ' + name + 'at ' + pdf_path)

1 PDF written for AMIT SINGH_20316at gdrive/My Drive/Colab Notebooks/NanoDev Speakers pdf Certificates/AMIT SINGH_20316.pdf
2 PDF written for AYAN KARMAKAR_20319at gdrive/My Drive/Colab Notebooks/NanoDev Speakers pdf Certificates/AYAN KARMAKAR_20319.pdf
3 PDF written for D. S. RAWAL_20315at gdrive/My Drive/Colab Notebooks/NanoDev Speakers pdf Certificates/D. S. RAWAL_20315.pdf
4 PDF written for SANTANU MAHAPTRA_20317at gdrive/My Drive/Colab Notebooks/NanoDev Speakers pdf Certificates/SANTANU MAHAPTRA_20317.pdf
5 PDF written for JAWAR SINGH_20318at gdrive/My Drive/Colab Notebooks/NanoDev Speakers pdf Certificates/JAWAR SINGH_20318.pdf
6 PDF written for YOGESH CHAUHAN_20314at gdrive/My Drive/Colab Notebooks/NanoDev Speakers pdf Certificates/YOGESH CHAUHAN_20314.pdf
7 PDF written for ARUN ASHOK_20320at gdrive/My Drive/Colab Notebooks/NanoDev Speakers pdf Certificates/ARUN ASHOK_20320.pdf
8 PDF written for H. S. JATANA_20313at gdrive/My Drive/Colab Notebooks/NanoDev Speakers pdf Certificat